In [7]:
# Install dependencies
!pip install textblob
!pip install -U scikit-learn
!pip install nrclex
from google.colab import drive

import sys
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from textblob import TextBlob
from nrclex import NRCLex
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
# Download the necessary NLTK data
nltk.download('punkt_tab') # Downloading the missing punkt_tab resource.


drive.mount('/content/drive', force_remount=True)
drive_libs_path = '/content/drive/MyDrive/libs'
sys.path.append(drive_libs_path)

# Assuming `CombinedAcceptanceIndexer` is already defined elsewhere
from usage_examples import get_acceptance_indexes, CombinedAcceptanceIndexer
from noise_filter import NoiseFilter

def mount_drive():
    drive.mount('/content/drive', force_remount=True)

def load_and_filter_data(file_path):
    df = pd.read_csv(file_path, low_memory=False)
    df.rename(columns={'comments_orig': 'comments'}, inplace=True)
    df['clean_title'] = df['clean_title'].astype(str).str.lower()
    df['comments'] = df['comments'].astype(str)
    df['separated_comment'] = ''
    return df

def get_sentiment(text,method='textblob'):
    if method == 'textblob':
      blob = TextBlob(text)
      return blob.sentiment.polarity, blob.sentiment.subjectivity
    elif method == 'vader':
        analyzer = SentimentIntensityAnalyzer()
        scores = analyzer.polarity_scores(text)
        return scores['compound'], scores['compound']
    elif method == 'nrc':
        emotion_analyzer = NRCLex(text)
        sentiment_scores = emotion_analyzer.affect_frequencies
        polarity = sentiment_scores.get('positive', 0) - sentiment_scores.get('negative', 0)
        subjectivity = sum(sentiment_scores.values()) - sentiment_scores.get('anticipation', 0) - sentiment_scores.get('trust', 0)
        return polarity, subjectivity
    else:
        raise ValueError(f"Invalid method: {method}. Choose from 'textblob', 'vader', or 'nrc'.")

def process_comments(row, method='textblob'):
    try:
        comment = row['comments']  # Get the 'comments' string directly
    except AttributeError:
        return 0, 0  # Handle cases where 'comments' column is missing or invalid

    # Call get_sentiment with the specified method
    polarity, subjectivity = get_sentiment(str(comment), method=method)

    return polarity, subjectivity

def preprocess_data(df,method='textblob'):
    df['separated_comment'] = df['comments'].apply(lambda x: x.split('|__|'))
    df['comments_polarity'], df['comments_subjectivity'] = zip(*df.apply(lambda row: process_comments(row, method=method), axis=1))
    df['clean_title_polarity'], df['clean_title_subjectivity'] = zip(*df['clean_title'].apply(lambda title: get_sentiment(title, method=method)))
    return df

def prepare_features_and_labels(df,method='textblob',base_feature=False):
     # Select features for the model
    base_features = ['clean_title', 'score', 'separated_comment', 'num_comments', 'upvote_ratio','comments_polarity', 'comments_subjectivity', 'clean_title_polarity',
            'clean_title_subjectivity']
    base_numerical_features = ['score', 'num_comments', 'upvote_ratio','comments_polarity', 'comments_subjectivity', 'clean_title_polarity',
            'clean_title_subjectivity']
    if base_feature:  # If base_feature is True, select only base features
        features = base_features
        numerical_features = base_numerical_features
    else:  # If base_feature is False, select base features + method-specific feature
        if method == 'textblob':
            features = base_features + ['TextBlobAcceptanceIndexer']
            numerical_features = base_numerical_features + ['TextBlobAcceptanceIndexer']
        elif method == 'nrc':
            features = base_features + ['NRCAcceptanceIndexer']
            numerical_features = base_numerical_features + ['NRCAcceptanceIndexer']
        elif method == 'vader':
            features = base_features + ['VADERAcceptanceIndexer']
            numerical_features = base_numerical_features + ['VADERAcceptanceIndexer']
        else:
            raise ValueError(f"Invalid method: {method}. Choose from 'textblob', 'vader', or 'nrc'.")

    X = df[features]
    X[numerical_features] = X[numerical_features].fillna(0)
    y = df['2_way_label'].astype(str)
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', 'passthrough', numerical_features),
            ('text', TfidfVectorizer(max_features=5000), 'clean_title')
        ])

    X_train = preprocessor.fit_transform(X_train)
    X_test = preprocessor.transform(X_test)

    return (X_train, X_test, y_train, y_test), label_encoder, preprocessor

def build_pipeline():
    # Using Logistic Regression for classification
    model = Pipeline([
        ('classifier', LogisticRegression(max_iter=500))
    ])
    return model

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
    print("Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

def enhanced_predict_new_comments(model, preprocessor, comments, title,method='textblob', base_feature=False):
    # Define base features
    base_features_df = pd.DataFrame({'clean_title': title * len(comments),
                                       'score': [0] * len(comments),
                                       'num_comments': [0] * len(comments),
                                       'upvote_ratio': [0] * len(comments),
                                       'separated_comment': [''] * len(comments),
                                       'comments_polarity': [0] * len(comments),  # Add comments_polarity
                                       'comments_subjectivity': [0] * len(comments),  # Add comments_subjectivity
                                       'clean_title_polarity': [0] * len(comments),  # Add clean_title_polarity
                                       'clean_title_subjectivity': [0] * len(comments)})

    # Add method-specific features if base_feature is False
    if not base_feature:
        if method == 'textblob':
            base_features_df['TextBlobAcceptanceIndexer'] = [0] * len(comments)
        elif method == 'vader':
            base_features_df['VADERAcceptanceIndexer'] = [0] * len(comments)
        elif method == 'nrc':
            base_features_df['NRCAcceptanceIndexer'] = [0] * len(comments)
        else:
            raise ValueError(f"Invalid method: {method}. Choose from 'textblob', 'vader', or 'nrc'.")

    # Preprocess the new comments using the same preprocessor
    comments_transformed = preprocessor.transform(base_features_df)
    # Make predictions
    predictions = model.predict(comments_transformed)

    results = []
    for comment, pred in zip(comments, predictions):
        blob = TextBlob(comment)
        results.append({
            'comment': comment,
            'prediction': pred
        })

    return results

def main():
    mount_drive()
    file_path = "/content/drive/My Drive/datasets/data_with_indexers_applied_to_original_data_31_03_2025.csv"
    df = load_and_filter_data(file_path)
    processed_df = preprocess_data(df,method='nrc')
    (X_train, X_test, y_train, y_test), label_encoder, preprocessor = prepare_features_and_labels(processed_df,method='nrc',base_feature=True)

    model = build_pipeline()
    model.fit(X_train, y_train)

    evaluate_model(model, X_test, y_test)

    new_title = "Shocking Discovery Leaves Internet Divided—Hoax or Breakthrough?"
    new_comments = [
        "This is outrageous! There's no way this can be true!",
        "I can't believe people are falling for this. It's just a hoax.",
        "Great news! This is exactly what we needed.",
        "Interesting perspective, I hadn't considered that before."
    ]

    predictions = enhanced_predict_new_comments(model, preprocessor, new_comments, new_title,method='nrc', base_feature=True)
    print("\nEnhanced Predictions with Sentiment Analysis:")
    for result in predictions:
        print(f"\nComment: {result['comment']}")
        print(f"Prediction: {result['prediction']}")

if __name__ == "__main__":
    main()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Mounted at /content/drive
Mounted at /content/drive


<ipython-input-7-eaedc1b16484>:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[numerical_features] = X[numerical_features].fillna(0)


Accuracy: 0.85
Confusion Matrix:
[[ 1619  1698]
 [  459 10383]]
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.49      0.60      3317
           1       0.86      0.96      0.91     10842

    accuracy                           0.85     14159
   macro avg       0.82      0.72      0.75     14159
weighted avg       0.84      0.85      0.83     14159


Enhanced Predictions with Sentiment Analysis:

Comment: This is outrageous! There's no way this can be true!
Prediction: 1

Comment: I can't believe people are falling for this. It's just a hoax.
Prediction: 1

Comment: Great news! This is exactly what we needed.
Prediction: 1

Comment: Interesting perspective, I hadn't considered that before.
Prediction: 1


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
